In [16]:
import pandas as pd
from transformers import BertTokenizer

In [2]:
train = pd.read_csv('../data/tweet-sentiment-extraction/train.csv')

In [18]:
tokenizer = BertTokenizer.from_pretrained('../../bert_models/large_uncased_wwm/vocab.txt')

In [3]:
train.shape

(27486, 4)

In [9]:
train.dropna(subset=['text','selected_text'], how='any', inplace=True)

In [10]:
train.shape

(27485, 4)

In [60]:
train.head()

,textID,text,selected_text,sentiment,start_pos,end_pos,tokens,start,end,invert_map
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral,55,98,"[spent, the, entire, morning, in, a, meeting, ...",13,25,"[0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 10, 11, 1..."
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive,4,8,"[oh, !, good, idea, about, putting, them, on, ...",2,2,"[0, 0, 1, 2, 3, 4, 5, 6, 7, 8]"
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral,0,43,"[says, good, (, or, should, i, say, bad, ?, ),...",0,11,"[0, 1, 2, 2, 3, 4, 5, 6, 6, 6, 7, 7, 8, 8, 8, ..."
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative,0,34,"[i, don, ##t, think, you, can, vote, anymore, ...",0,8,"[0, 1, 1, 2, 3, 4, 5, 6, 6, 7, 8]"
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive,5,11,"[ha, ##ha, better, drunken, t, ##wee, ##ting, ...",2,2,"[0, 0, 1, 2, 3, 3, 3, 4, 5, 5]"


In [47]:
train['text'] = train['text'].apply(lambda x: x.strip())

In [48]:
train['start_pos'] = train.apply(lambda x: x['text'].find(x['selected_text']), axis=1)
train['end_pos'] = train.apply(lambda x: x['start_pos']+len(x['selected_text']), axis=1)

In [49]:
train['start_pos'].describe()

count    27485.000000
mean        15.604839
std         26.179125
min          0.000000
25%          0.000000
50%          0.000000
75%         23.000000
max        134.000000
Name: start_pos, dtype: float64

In [1]:
data = []
for text, sp, ep in zip(train['text'].tolist(), train['start_pos'].tolist(), train['end_pos'].tolist()):
    split_text = text.split()
    tokens, labels, invert_map = [], [], []
    cur_length = 0
    for idx, w in enumerate(split_text):
        if cur_length+idx+len(w)>=sp and cur_length+idx<ep:
            started = True# space
        else:
            started = False
        for token in tokenizer.tokenize(w):
            tokens.append(token)
            invert_map.append(idx)
            if started:
                labels.append(len(tokens)-1)
        cur_length+=len(w)
    data.append((tokens, min(labels), max(labels), invert_map, tokens[min(labels)],tokens[max(labels)]))
#     if len(data)>5:
#         break

NameError: name 'train' is not defined

In [55]:
train.loc[0, 'text'].split()[data[0][3][25]]

'fun.'

In [53]:
data[:5]

[(['spent',
   'the',
   'entire',
   'morning',
   'in',
   'a',
   'meeting',
   'w',
   '/',
   'a',
   'vendor',
   ',',
   'and',
   'my',
   'boss',
   'was',
   'not',
   'happy',
   'w',
   '/',
   'them',
   '.',
   'lots',
   'of',
   'fun',
   '.',
   'i',
   'had',
   'other',
   'plans',
   'for',
   'my',
   'morning'],
  13,
  25,
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   7,
   8,
   9,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   16,
   17,
   17,
   18,
   19,
   20,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27],
  'my',
  '.'),
 (['oh',
   '!',
   'good',
   'idea',
   'about',
   'putting',
   'them',
   'on',
   'ice',
   'cream'],
  2,
  2,
  [0, 0, 1, 2, 3, 4, 5, 6, 7, 8],
  'good',
  'good'),
 (['says',
   'good',
   '(',
   'or',
   'should',
   'i',
   'say',
   'bad',
   '?',
   ')',
   'afternoon',
   '!',
   'http',
   ':',
   '/',
   '/',
   'pl',
   '##ur',
   '##k',
   '.',
   'com',
   '/',
   'p',
   '/',
   'w',
   '##x'

In [56]:
tokens, start, end, invert_map, _, _ = zip(*data)

In [57]:
len(tokens)

27485

In [58]:
train['tokens'] = tokens

In [59]:
train['start'] = start
train['end'] = end
train['invert_map'] = invert_map

In [73]:
senti2label = {
    'positive':2,
    'negative':0,
    'neutral':1
}
train['senti_label']=train['sentiment'].apply(lambda x: senti2label[x])

In [74]:
train.reset_index(drop=True, inplace=True)

In [75]:
train.head()

,textID,text,selected_text,sentiment,start_pos,end_pos,tokens,start,end,invert_map,senti_label,fold
0,a3d0a7d5ad,Spent the entire morning in a meeting w/ a ven...,my boss was not happy w/ them. Lots of fun.,neutral,55,98,"[spent, the, entire, morning, in, a, meeting, ...",13,25,"[0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 10, 11, 1...",1,0
1,251b6a6766,Oh! Good idea about putting them on ice cream,Good,positive,4,8,"[oh, !, good, idea, about, putting, them, on, ...",2,2,"[0, 0, 1, 2, 3, 4, 5, 6, 7, 8]",2,0
2,c9e8d1ef1c,says good (or should i say bad?) afternoon! h...,says good (or should i say bad?) afternoon!,neutral,0,43,"[says, good, (, or, should, i, say, bad, ?, ),...",0,11,"[0, 1, 2, 2, 3, 4, 5, 6, 6, 6, 7, 7, 8, 8, 8, ...",1,0
3,f14f087215,i dont think you can vote anymore! i tried,i dont think you can vote anymore!,negative,0,34,"[i, don, ##t, think, you, can, vote, anymore, ...",0,8,"[0, 1, 1, 2, 3, 4, 5, 6, 6, 7, 8]",0,0
4,bf7473b12d,haha better drunken tweeting you mean?,better,positive,5,11,"[ha, ##ha, better, drunken, t, ##wee, ##ting, ...",2,2,"[0, 0, 1, 2, 3, 3, 3, 4, 5, 5]",2,0


In [76]:
from sklearn.model_selection import StratifiedKFold

In [77]:
kf = StratifiedKFold(n_splits=5)

In [78]:
train['fold'] = 0
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train['senti_label'])):
    train.loc[valid_idx, 'fold'] = fold

In [79]:
train['fold'].value_counts()

0    5499
1    5498
4    5496
3    5496
2    5496
Name: fold, dtype: int64

In [80]:
train.to_pickle('../data/train.pkl')

In [83]:
tokenizer.tokenize('neutral')

['neutral']